## Client Side: Customized Job-level Auhtorization

Let's take a look at authorization on the client side.

**Setup**

* `server`: NVFlare server
* `site-1`: Site-1 has a CustomSecurityHandler set up which does not allow the job "secret-job" to run. All other jobs will be able to deploy and run on site-1.
* `site-2`: Site-2 allows any job to be deployed and run.

**Expectation**
* "secret-job" will be deployed and run on site-2 but not on site-1


What we will do: 

* Install dependencies
* Download data
* Generate two job configs using fl_jobs.py



In [ ]:
# install dependencies

! pip install -r code/requirements.txt

# download data

! python code/data/download.py

In [ ]:
%cd code
! python fl_jobs.py
# change back
%cd - 

Next, we will:
* Create a POC workspace
* Install the customized security handler to site-1
* Edit site-1/local/resources.json to add security handler component 
> Note: 
  To simplify, we just copy the pre-edited resources.json to that location

In [ ]:
# prepare poc
! echo y | nvflare poc prepare -n 2

In [ ]:
# cp security handler and component config
!cp -r security/site-1/* /tmp/nvflare/poc/example_project/prod_00/site-1/local/.

Now we are ready to run the job.

* Start POC

    Use a terminal (not notebook cell) to start the POC with the following command:

```
    nvflare poc start -ex admin@nvidia.com 

```

    This will bring up the FL system.


In [ ]:
#  Submit jobs
#  Assuming at this point FL system is already running via poc start command

!  nvflare job submit -j /tmp/nvflare/jobs/workdir/fedavg

# The job should finish as expected

The FedAvg job completed successfully. Now let's submit the "secret-job". In this step, we’ll use the FLARE Python API instead of the CLI Job command to showcase a different type of API.


In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_secure_session
 
username = "admin@nvidia.com"
admin_user_dir = os.path.join("/tmp/nvflare/poc/example_project/prod_00/", username)

sess = new_secure_session(username=username, startup_kit_location=admin_user_dir)
print(sess.get_system_info())

In [ ]:
# !  nvflare job submit -j /tmp/nvflare/jobs/workdir/secret-job


job_path = "/tmp/nvflare/jobs/workdir/secret-job"
job_id = sess.submit_job(job_path)
print(job_id + " was submitted")

Job will fail to schedule, you should see some of the following in terminal (not on notebook)

```
2025-10-12 13:14:31,496 - site_security - ERROR - Authorization failed. Reason: Not authorized to execute: check_resources
2025-10-12 13:14:31,499 - ServerEngine - ERROR - Client reply error: Not authorized to execute: check_resources
2025-10-12 13:14:31,502 - DefaultJobScheduler - INFO - [identity=server, run=?] - Try to schedule job a877c175-35f4-4376-ae3e-663e269ec84e, get result: (not enough sites have enough resources (ok sites 1 < min sites 2)).
2025-10-12 13:14:31,504 - site_security - ERROR - Authorization failed. Reason: Not authorized to execute: check_resources
2025-10-12 13:14:31,505 - ServerEngine - ERROR - Client reply error: Not authorized to execute: check_resources
2025-10-12 13:14:31,508 - DefaultJobScheduler - INFO - [identity=server, run=?] - Try to schedule job d404b9ec-453d-4184-8c84-2dfe0816979a, get result: (not enough sites have enough resources (ok sites 1 < min sites 2)).

```
Since it will not terminate, we will just aborf the job. 


In [ ]:
sess.abort_job(job_id)

**Cleanup**

In [ ]:
! nvflare poc stop

In [ ]:
! nvflare poc clean